In [19]:
import pandas as pd
from kalm_benchmark.evaluation.evaluation import load_benchmark, Col

In [20]:
df_bench = load_benchmark(with_categories=True)

In [21]:
def get_base_check_id(check_id: str) -> str:
    if check_id.count('-') == 1:
        # no variant present -> check_id is base_check_id
        return check_id
    base_id, variant = check_id.rsplit("-", maxsplit=1)
    return base_id

In [22]:
def get_pretty_name(r: pd.Series):
    name = r[Col.Name].replace(f"{r[Col.CheckId].lower()}-", "")
    return name.replace("-", " ").title()

In [23]:
PRETTY_NAME = "pretty_name"
df_bench[PRETTY_NAME] = df_bench.apply(get_pretty_name, axis=1)

In [24]:
BASE_CHECK_ID = "base_check_id"
df_bench[BASE_CHECK_ID] = df_bench.check_id.map(get_base_check_id)

In [25]:
df_bench.head()

,check_id,name,expected,description,path_to_check,category,pretty_name,base_check_id
0,RES-007-0,res-007-0-no-limitrange-object-for-namespace,alert,None,LimitRange.metadata.namespace|.metadata.namespace,Reliability,No Limitrange Object For Namespace,RES-007
1,RES-007-1,res-007-1-no-default-cpu-request-for-namespace,alert,None,LimitRange.spec.limits.defaultRequest.cpu|.spe...,Reliability,No Default Cpu Request For Namespace,RES-007
2,RES-007-2,res-007-2-no-default-cpu-limits-for-namespace,alert,None,LimitRange.spec.limits.default.cpu|.spec.limit...,Reliability,No Default Cpu Limits For Namespace,RES-007
3,RES-007-3,res-007-3-no-cpu-limits-for-namespace,alert,None,LimitRange.spec.limits.min.cpu|LimitRange.spec...,Reliability,No Cpu Limits For Namespace,RES-007
4,RES-008-1,res-008-1-no-default-memory-request-for-namespace,alert,None,LimitRange.spec.limits.defaultRequest.memory|....,Reliability,No Default Memory Request For Namespace,RES-008


In [26]:
df_overview = df_bench.groupby(by=[Col.Category, BASE_CHECK_ID]).aggregate(name=pd.NamedAgg(PRETTY_NAME, "first"), num_variants= pd.NamedAgg(Col.CheckId, 'count'))

In [27]:
pd.options.display.max_rows = 1000
print(df_overview.to_latex(caption="An overview of all generated checks"))

\begin{table}
\caption{An overview of all generated checks}
\begin{tabular}{lllr}
\toprule
 &  & name & num_variants \\
category & base_check_id &  &  \\
\midrule
DataSecurity & CM-001 & Sensitive Key Referenced In Configmap & 1 \\
\cline{1-4}
\multirow[t]{17}{*}{IAM} & RBAC-001 & Use Cluster Admin Role & 2 \\
 & RBAC-002 & Read Access To Secrets & 6 \\
 & RBAC-003 & Role Use Resource Wildcard & 6 \\
 & RBAC-004 & Role Creates Pods & 2 \\
 & RBAC-005 & Role Attaches To Pods & 2 \\
 & RBAC-006 & Role Exec Into Pods & 2 \\
 & RBAC-007 & Role Binds Default Serviceaccount & 2 \\
 & RBAC-008 & Role Port Forward Pods & 6 \\
 & RBAC-009 & Role Impersonation & 2 \\
 & RBAC-010 & Role Manages Rbac & 8 \\
 & RBAC-012 & Role Info Disclosure & 6 \\
 & RBAC-013 & Role Destructive & 4 \\
 & RBAC-014 & Role Event Deletion & 4 \\
 & RBAC-015 & Role Core Dns Poisoning & 4 \\
 & RBAC-016 & Serviceaccount Without Binding & 1 \\
 & RBAC-017 & Too Many Roles Per Subject & 1 \\
 & RBAC-020 & Role Manages Rb

In [28]:
df_overview


name  \
category     base_check_id                                                      
DataSecurity CM-001                     Sensitive Key Referenced In Configmap   
IAM          RBAC-001                                  Use Cluster Admin Role   
             RBAC-002                                  Read Access To Secrets   
             RBAC-003                              Role Use Resource Wildcard   
             RBAC-004                                       Role Creates Pods   
             RBAC-005                                   Role Attaches To Pods   
             RBAC-006                                     Role Exec Into Pods   
             RBAC-007                       Role Binds Default Serviceaccount   
             RBAC-008                                  Role Port Forward Pods   
             RBAC-009                                      Role Impersonation   
             RBAC-010                                       Role Manages Rbac   
             RBAC-012                                    Role Info Disclosure   
             RBAC-013                                        Role Destructive   
             RBAC-014                                     Role Event Deletion   
             RBAC-015                                 Role Core Dns Poisoning   
             RBAC-016                          Serviceaccount Without Binding   
             RBAC-017                              Too Many Roles Per Subject   
             RBAC-020                                       Role Manages Rbac   
Misc         PSA-001               No Pod Security Admission Label Configured   
             PSA-002        Using Privileged Privilegedpodsecurity Level I...   
             PSA-003        Just Warning About Violations Of The Pod Secur...   
Network      NP-001                          Namespace Without Network Policy   
             NP-002                                   Only Ingress Is Blocked   
             NP-003         Network Policy Allows Access To Cloud Metadata...   
             NP-004                   Network Policy Allows Access To Kubelet   
             NP-005                   Network Policy Refers No Valid Workload   
Reliability  REL-001                                        No Readinessprobe   
             REL-002                                         No Livenessprobe   
             REL-003                                         No Priorityclass   
             REL-004                              No Node Selection Specified   
             RES-001                   Default Memory Requests From Namespace   
             RES-002                     Default Memory Limits From Namespace   
             RES-003                      Default Cpu Requests From Namespace   
             RES-004                        Default Cpu Limits From Namespace   
             RES-005        Default Ephemeral Storage Requests From Namespace   
             RES-006          Default Ephemeral Storage Limits From Namespace   
             RES-007                       No Limitrange Object For Namespace   
             RES-008                  No Default Memory Request For Namespace   
             RES-009                   No Resource Quota Applied To Namespace   
Workload     NS-001                                     Use Default Namespace   
             NS-002                                  Place Pod In Kube System   
             POD-002                                      Explicit Default Sa   
             POD-003                                       Default Pod And Sa   
             POD-008                                           Use No Hostpid   
             POD-009                                           Use No Hostipc   
             POD-010                                       Use No Hostnetwork   
             POD-011                                            Uses Hostport   
             POD-012                Use Hostaliases To Modify Pods  Etc Hosts   
             POD-013   